# Information need to save in advance
- Original Image
- pos map
- vertices
- camera_matrix, pose, rotationmatrix


In [4]:
import cv2
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../../PRNet")
from api import PRN
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '' # cpu
prn = PRN(is_dlib = True, prefix="../../PRNet")
from utils.render import render_texture
from utils.rotate_vertices import frontalize
from utils.estimate_pose import estimate_pose
import numpy as np

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Please use `layer.__call__` method instead.





INFO:tensorflow:Restoring parameters from ../../PRNet/Data/net-data/256_256_resfcn256_weight


In [5]:
import pickle
import time
def save_information(img, output):
    pos = prn.process(img) 
    vertices = prn.get_vertices(pos)
    cam_mat, pose, R = estimate_pose(vertices)

    dictionary = {'img': img, 'pos':pos, 'vertices': vertices,
                  'cam_mat': cam_mat, 'pose':pose, 'R': R}
    pickle.dump(dictionary,open(output, 'wb'))
    print("Finish dump one file")

t1 = time.time()
save_information(cv2.imread('/home/vuthede/Desktop/3D/31_in.png'), "tmp.pickle")
print("Time:", time.time()-t1)


Here is input:  (565, 565, 3)
Finish dump one file
Time: 3.2019810676574707


In [12]:
def load_information(file):
    return pickle.load(open(file, 'rb'))

# info = load_information("tmp.pickle")
# info.keys()
# texture_ref = cv2.remap(info['img']/255.0, info['pos'][:,:,:2].astype(np.float32), None, interpolation=cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT,borderValue=(0))

# [h, w, c] = info['img'].shape
    
# color = prn.get_colors_from_texture(texture_ref)
    
# new_image = render_texture(info['vertices'].T, color.T, prn.triangles.T, h, w, c = 3)

In [5]:
import pickle
import time
import os

def save_information(img, output):
    pos = prn.process(img) 
    vertices = prn.get_vertices(pos)
    cam_mat, pose, R = estimate_pose(vertices)

    dictionary = {'img': img, 'pos':pos, 'vertices': vertices,
                  'cam_mat': cam_mat, 'pose':pose, 'R': R}
    pickle.dump(dictionary,open(output, 'wb'))

def save_3d_info_sync(video_synthesis, output="./synthesis"):
    if  not os.path.isdir(output):
        os.makedirs(output)

    obama_syn = cv2.VideoCapture(video_synthesis)
    x1 = 685
    y1 = 85
    x2 = 1250
    y2 = 650

    i = 0
    while True:
        i+=1
        for j in range(3):
            ret, img_obama_syn = obama_syn.read()

        if not ret:
            print(f"Break at frame {i}")
            break
        
        img_obama_syn = cv2.resize(img_obama_syn, (x2-x1, y2-y1))
        save_information(img_obama_syn,os.path.join(output, f'{i}.pickle' ))
        print(f"Finish dump one file {output}/{i}.pickle")


def save_3d_info_HD(video_HD, output="./HD"):
    if  not os.path.isdir(output):
        os.makedirs(output)

    obama_fullhd = cv2.VideoCapture(video_HD)
    x1 = 685
    y1 = 85
    x2 = 1250
    y2 = 650

    i = 0
    while True:
        i+=1
        for j in range(3):
            ret1, img_obamahd = obama_fullhd.read()

        if not ret1 or i>= 43*30:  # thus frame obama change view
            print(f"Break at frame {i}")
            break
        
        obama_crop = img_obamahd[y1:y2, x1:x2]
        save_information(obama_crop,os.path.join(output, f'{i}.pickle' ))
        print(f"Finish dump one file {output}/{i}.pickle")




In [6]:
save_3d_info_HD("/home/vuthede/AI/first-order-model/obama_fullhd.mp4")

Here is input:  (565, 565, 3)
Finish dump one file ./HD/1.pickle
Here is input:  (565, 565, 3)
Finish dump one file ./HD/2.pickle
Here is input:  (565, 565, 3)
Finish dump one file ./HD/3.pickle
Here is input:  (565, 565, 3)
Finish dump one file ./HD/4.pickle
Here is input:  (565, 565, 3)
Finish dump one file ./HD/5.pickle
Here is input:  (565, 565, 3)


KeyboardInterrupt: 

In [ ]:
def load_information(file):
    return pickle.load(open(file, 'rb'))

info = load_information("./HD/4.pickle")
info.keys()
texture_ref = cv2.remap(info['img']/255.0, info['pos'][:,:,:2].astype(np.float32), None, interpolation=cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT,borderValue=(0))

[h, w, c] = info['img'].shape
    
color = prn.get_colors_from_texture(texture_ref)
    
new_image = render_texture(info['vertices'].T, color.T, prn.triangles.T, h, w, c = 3)